# Ride-Sharing Autonomous Vehicle Fleet Optimization
## CS/ECE/ISyE 524 — Introduction to Optimization
## Spring 2017


### Dylan Goss (dgoss28@gmail.com) and Patrick Roberts (pgroberts21@yahoo.com)

## Table Of Contents

#### 1. <a href="#Introduction">Introduction</a>
#### 2. <a href="#Mathematical-Model">Mathematical Model</a>
#### 3. <a href="#Solution">Solution</a>
<ul><li><a href="#Adjustable-Parameters-Used-Throughout-Iterations">Parameters</a></li>
<li><a href="#Iteration-1:-A-Car-always-by-its-Owner">Iteration 1</a></li>
<li><a href="#Iteration-2:-Considering-Other-Customers---Minimizing-Distance-to-All-Customers">Iteration 2</a></li>
<li><a href="#Iteration-3:-Considering-Other-Customers---Maximizing-Amount-of-Customers-within-Radius-of-Car">Iteration 3</a></li>
</ul>
#### 4. <a href="#Results-And-Discussion"> Results And Discussion</a>
#### 5. <a href="#Conclusion"> Conclusion </a>


## Introduction

<img src = "https://i.wheelsage.org/pictures/tesla/logotypes/autowp.ru_tesla_logo_3.jpg", style="height:300px;width:300px;"></img>

This project will optimize the spacing of the Tesla Fleet with relation to distance from car owner and other Tesla vehicles to maximize the amount of topical city map that can be covered, minimize the distance to a customer, or maximize the amount of customers the fleet can serve.

As more Tesla vehicles are introduced to city streets and autonomous driving becomes safe enough to be passed by legislature to become a normal activity, Tesla cars may be added to the Tesla Fleet. As described by Elon Musk in his second master plan, the Tesla fleet will consist of cars that will have the availability to act as a taxi or delivery service for other corporations that may want to use them. (electrek.co) This means owners of Tesla will be able receive income on their car when they are not using it. In order to maximize this profit, the cars will be optimally spaced from one another to cover more area of use. The cars also must remain within a reasonable distance of the owner so it can be “picked up” by their owner at their own discretion. The team plans to use optimization regularizations to put emphasis on these different components of this objective. 

The data for this formulation will be based off distances on a map. Initially, the team will analyze a simple grid to begin formulation and then larger grids that will be synthetically generated to allow for the Fleets location to change as the map size changes and the number of cars changes. The following report will have a mathematical model to outline the variables, constraints and objective functions of different models to solve the different problems. Next, a JuMP code will be written to solve the model and discuss the different components that change as it grows. Lastly, we will discuss the findings and give recommendations for how Tesla can use our model to help them formulate the placement of cars in their fleet.  

<b>Citations:</b>

https://electrek.co/2016/07/20/teslashared-fleet-autonomous-fleet-money-off-their-car/

https://www.tesla.com/blog/master-plan-part-deux

## Mathematical Model
#### Assumptions
- Cars can travel in a straight line from their starting point to another location
- The task that is performed when at the customer will not stray the vehicle to far from its owner
- All cars are full of charge or have enough to complete tasks throughout the time in the fleet


#### 2a. Constrain Owner Proximity (m1)
##### Decision Variables
- The decision variables being considered are the X and Y coordinates of a vehicle located on the city map. These will be numerical and not constrained by integer values along this grid. We will introduce a different amount of cars as the model progresses so the amount of these variables will change. The variables be denoted as:

$\left[
X_n , Y_n
\right]$

- Each of these locations will have a radial area that it covers on the map. The size of the radius will be an implemented parameter within the model, $r$. In essence, the car itself is the size of its radial coverage with the center point being the cars location. 

$ Area = \pi r^2 $


#####  Constraints
- The cars must remain on the grid based off the given size with grid size adjsuting by parameter, $ p $. These constraints will look as follows:

$ 0 \le X_n \le p \\
 0 \le Y_n \le p $

- Vehicles must be within distance of the owner, given by $v$, to allow for pickup. The owner's will be a random variable due to the fact that they may go to different locations on the grid. The coordinates of the owner will be represented by $\left[A_n , B_n \right]$. And the distance contraint will be represented by:

$\sqrt{(X_n - A_n)^2 + (Y_n - B_n)^2} \le v\space for \space  n = 1,2, \ldots, n$

- Cars should not overlap their coverage circles, we will use the radius parameter, $r$, to assist in this constraint. This constraint will also not allow the cars to be in the same location as one another and will satisfy: $\left[X_1 , Y_1 \right] \ne \left[X_2 , Y_2 \right]\ne \ldots \ne \left[X_n , Y_n \right]$
 Also using the distance formula, the constraint for vehicles 1 and 2 not being within the radial distance of another is as follows:

$\sqrt{(X_1 - X_2)^2 + (Y_1 - Y_2)^2} \ge 2r$


##### Objective 
- The objective of this is to maximize the space that the fleet covers on a given map. We will use a regularization technique to allow for a tradeoff of space allowed to be covered by the vehicle and the distance that it can be placed from the driver. The maximization will allow for the cars to naturally spread along the grid to cover more area. We will make the vertices of the location of the cars to create the largest area of a polygon. The objective funciton will look as follows:

$Max \sum_{i=1}^{n}((x_1*y_2 - x_2*y_1) + (x_2*y_3 - x_3*y_2) + \ldots +(x_n*y_1 - x_1*y_n))$

#### Standard Form
$maximize \sum_{i=1}^{n}((x_1*y_2 - x_2*y_1) + (x_2*y_3 - x_3*y_2) + \ldots +(x_n*y_1 - x_1*y_n))$

$subject\space to$

$\sqrt{(x_n - x_n+1)^2 + (y_n - y_n+1)^2} -2r  \ge 0  \space for \space  n = 1,2, \ldots, n-1$

$-\sqrt{(x_n - a_n)^2 + (y_n - b_n)^2} + v \ge 0\space for \space  n = 1,2, \ldots, n$

$p \ge x_n \ge 0 $

$p \ge y_n \ge 0 $

$a_n = A_n$

$b_n = B_n$

#### 2b. Minimize Customer Distance Model (m2)
#####  Decision Variables and Parameters
- There is an additional parameter for the amount of customers that are in the city. This parameter will be two random variables that represent the X and Y coordinate of the customer. 
- There are no new decision variables because we are optimizing the location of the vehicle based off fixed, random points.


##### Constraints
- For this model, the constraints that did not allow for overlapping of coverage and the proximity to the ownner were removed. Ignoring the location of the owner is appropriate allowing the cars to go anywhere on the map.
- A nonlinear expression was added to calcualte the distance of the car the customer. It will look as follows:

$\sqrt{(X_n - C_n)^2 + (Y_n - D_n)^2} \space for \space  n = 1,2, \ldots, n$

##### Objective
- A new objective function from this model which aims to minimize the sum of the distance between the cars and the customers:

$min \sum_{i=1}^{n} \sqrt{(X_n - C_n)^2 + (Y_n - D_n)^2} $

##### Standard Form
$minimize\space \sum_{i=1}^{n} \sqrt{(X_n - C_n)^2 + (Y_n - D_n)^2}\space for \space  n = 1,2, \ldots, n $

$subject\space to$

$-\sqrt{(x_n - a_n)^2 + (y_n - b_n)^2} + v \ge 0\space for \space  n = 1,2, \ldots, n$

$p \ge x_n \ge 0 $

$p \ge y_n \ge 0 $

$c_n = C_n$

$d_n = D_n$

#### 2c.  Maximize Customer Demand Model (m3)
##### Variables
- The same variables will be used for this model, customer location and changing the location of the car.
##### Constraints
- The constraint of this problem is an integer, if the distance between the two points is less than the radius, then that corresponding z value is a 1.
##### Objective
- The new objective maximizes the amount of customers that fall within the range of the Tesla Fleet. This is modeled as follows:

$max \sum_{i=1}^{customers} z_i $

##### Standard Form
$minimize\space $

$subject\space to$

$p \ge x_n \ge 0 $

$p \ge y_n \ge 0 $

$c_n = C_n$

$d_n = D_n$




## Solution

### Adjustable Parameters Used Throughout Iterations

In [ ]:
## Parameters

n = 4 # Number of vehicles in the system
p = 10 # Size of grid
v = 1 # Distance Car can be from owner, v
r = 1.5 # Radius of coverage, r

# Randomize locations of owners
A = p*rand(n);
B = p*rand(n);

customers = 10; # Amount of customers in the city

# Randomize locations of customers
C = p*rand(customers);
D = p*rand(customers); 

### Iteration 1: A Car always by its Owner

Creating the basis of the model for which we will expand upon. This model starts with a simple restriction of distance that the owner may be from their vehicle when not currently using it.

In [ ]:
using JuMP,Ipopt

## The Model
m = Model(solver = IpoptSolver())

@variable(m, x[1:n] >= 0) # x coordinate of vehicle n
@variable(m, y[1:n] >= 0) # y coordinate of vehicle n
@variable(m, a[1:n] >= 0)
@variable(m, b[1:n] >= 0)

#To begin iterations
setvalue(x[1:n], rand(n))
setvalue(y[1:n], rand(n))

#Random X and Y coordinates of Owner location of car n
for i in 1:n
    @NLconstraint(m, A[i] <= a[i] <= A[i])    
    @NLconstraint(m, B[i] <= b[i] <= B[i])    
end

#Constraints
#Distance allowed the car can be from the owner of distance v
for i in 1:n
    @NLconstraint(m, sqrt((x[i]-a[i])^2 + (y[i]-b[i])^2) <= v)
end

#Vehicle coverage should not overlap their coverage area with other cars
for i in 1:n-1
   @constraint(m, (x[i+1]-x[i])^2 + (y[i+1]-y[i])^2 >= 2*r)
end

#Do not allow two cars to be in the same location
#Constraint may be handled above...

 
#Maximize area of vertices
@NLobjective(m, Max, sum(x[i]*y[i+1] - x[i+1]*y[i] for i = 1:n-1))   

function runIterationOne();
    @time(solve(m))
    println(solve(m))
    #Calculation for Distance between owner and car
    for i in 1:n
        r = sqrt((getvalue(x[i])-getvalue(a[i]))^2 + (getvalue(y[i])-getvalue(b[i]))^2)
    end

    #Printing Results
    for i in 1:n
        println("Car ", i)
        println("Xcoord: ", getvalue(x[i]), "  Ycoord: ", getvalue(y[i]))
        println("Owner ", i)
        println("Xcoord: ", getvalue(a[i]), "  Ycoord: ", getvalue(b[i]))
        println("Distance between is ", r)
        println()
    end
end

### Iteration 2: Considering Other Customers - Minimizing Distance to All Customers

We want to maximize the number of customers that can be supported at any given time while keeping the cars close in distance to their owners. This could be seen as a convenience model i.e the owner will more than likely have their car around ready to go.

In [ ]:


using JuMP,Ipopt

## The Model
m2 = Model(solver = IpoptSolver())

@variable(m2, x[1:n] >= 0) # x coordinate of vehicle n
@variable(m2, y[1:n] >= 0) # y coordinate of vehicle n
@variable(m2, a[1:n] >= 0)
@variable(m2, b[1:n] >= 0)
@variable(m2, c[1:customers] >= 0)
@variable(m2, d[1:customers] >= 0)

#To begin iterations
setvalue(x[1:n], rand(n))
setvalue(y[1:n], rand(n))

#Random X and Y coordinates of Owner location of car n
for i in 1:n
@NLconstraint(m2, A[i] <= a[i] <= A[i])    
@NLconstraint(m2, B[i] <= b[i] <= B[i])    
end

for i in 1:customers
@NLconstraint(m2, C[i] <= c[i] <= C[i])    
@NLconstraint(m2, D[i] <= d[i] <= D[i])    
end


#Constraints
#Distance allowed the car can be from the owner of distance v
for i in 1:n
@NLconstraint(m2, sqrt((x[i]-a[i])^2 + (y[i]-b[i])^2) <= v)
end

@NLexpression(m2, dist[i=1:n,j=1:customers], sqrt((x[i]-c[j])^2 + (y[i]-d[j])^2))

# #Vehicle coverage should not overlap their coverage area with other cars
# for i in 1:n-1
#    @constraint(m, (x[i+1]-x[i])^2 + (y[i+1]-y[i])^2 >= 2*r)
# end

#Do not allow two cars to be in the same location
#Constraint may be handled above...

# Minimize the distances between the cars and potential customers
# for j=1:customers
#     @NLexpression(m2, dist[j], sum( sqrt( (C[j]-x[i])^2 + (D[j]-y[i])^2 ) for i=1:n))
# end


@NLobjective(m2, Min, sum(dist[i,j] for i=1:n,j=1:customers))

function runIterationTwo();
    @time(solve(m2))
    println(solve(m2))
    println("Total distance ", getobjectivevalue(m2))
    #Calculation for Distance between owner and car
    for i in 1:n
        r = sqrt((getvalue(x[i])-getvalue(a[i]))^2 + (getvalue(y[i])-getvalue(b[i]))^2)
    end

    #Printing Results
    for i in 1:n
        println("Car ", i)
        println("Xcoord: ", getvalue(x[i]), "  Ycoord: ", getvalue(y[i]))
        println("Owner ", i)
        println("Xcoord: ", getvalue(a[i]), "  Ycoord: ", getvalue(b[i]))
        println("Distance between is ", r)
        println()
    end
end

### Iteration 3: Considering Other Customers - Maximizing Amount of Customers within Radius of Car

We want to maximize the number of customers that can be supported at any given time while not worrying about the distance to owners. This could be seen as a profit maximization model i.e more customers are served.

In [ ]:
using JuMP,Gurobi
r = 1
customerLocation = p*rand(2,customers)

## The Model
m3 = Model(solver=GurobiSolver(OutputFlag=0));
@variable(m3, z[1:customers,1:n], Bin) # there is a car near customer n
@variable(m3, x[1:n,1:2] >= 0)
for i=1:n
    for j=1:customers
        @constraint(m3, norm(customerLocation[:,j]-x[i,:])-r <= -r*(1-z[j,i]) )
    end
end    
@objective(m3, Max, sum(z[i] for i in 1:customers))
            
            
function runIterationThree();
    @time(solve(m3))
    println(solve(m3))
    println("Total supported ", getobjectivevalue(m3))
    println("Customers supported ", getvalue(y[i]))
end


## Results and Discussions

### Helper Functions

In [ ]:
## Plotting the Solution
using PyPlot

function circlePlot(x,y,a,b,customersInvolved,locations);
    t = linspace(0,2π,100)
    #Plot of Car's position with red dot
    plot(getvalue(x[:]),getvalue(y[:]),"r.", label = "Car's Position") #Shows the car's position

    #Plot of owner's position with blue dot
    plot(getvalue(a[:]),getvalue(b[:]),"b.", label = "Owner's Position") #Shows the car's position

    #Plot radius car needs to be within owner
    for i in 1:n

        plot(getvalue(a[i])+ v*cos(t),getvalue(b[i])+ v*sin(t),"b")
    end

    #Plot radius of car's coverage on map
    for i in 1:n
        plot(getvalue(x[i])+r*cos(t),getvalue(y[i])+r*sin(t), "r")
    end
    
    if customersInvolved
        plot(locations[:,:][1],locations[:,:][2],"g.", label = "Customer's Position"); #Shows the customer's position
    end
    
    axis("equal")
    axis((0,10,0,10));
    grid("on")
    legend(loc="best");
end
;

### Iteration 1: A Car always by its Owner

In [ ]:
runIterationOne();
circlePlot(x,y,a,b,false,C,D);

Our base iteration sets the standard for which our model develops from. In this first iteration, we constrain the cars to a 1 unit radius of their owners. This allows quick availability for an owner that may need their car immediately. The parameter of the distance of the vehicle itself can be changed based on an owner's preference. This model has its obvious limitations in that it doesn't necessarily have an optimal position for the cars, it just ensures that they're within range.

### Iteration 2: Considering Other Customers - Minimizing Distance to All Customers

In [ ]:
runIterationTwo();
circlePlot(x,y,a,b,true,C,D);

In this example we see how the cars -- still under the constraint of being nearest to their owners -- move. This iteration attempts to minimize the overall distance between the vehicles and customers. This is a generalized summation, meaning that a car may sacrifice the availability to customers within its radius to be placed in an area that is more effecient for the overall customer population. This model is prone to longer optimization times based on the overall size of the customer population.

### Iteration 3: Considering Other Customers - Maximizing Amount of Customers within Radius of Car

In [ ]:
runIterationThree();

using PyPlot
t = linspace(0,2π,100)
axis("equal")
grid("on")
#Plot of Car's position with red dot

plot(getvalue(x[:,1]),getvalue(x[:,2]), "r.", label = "Car's Position"); #Shows the car's position
for i=1:customers
    plot(customerLocation[1,i],customerLocation[2,i], "b.")
end



This iteration let's us see what it's like when we don't focus on the restrictions of ownership and focus on the potential for profit, or the ability to stay next to customers. This model features integer constraints that tell whether or not a customer is in range. This model was limited obviously to the number of cars, so potentially showing the effects that supply has on demand could be a way to iterate this process in the future.

## Conclusion

This model was able to show us the ways in which inner-city car sharing and travel can be revolutionized. Though the levels of complexity and sophistication will undoubtedly rise as the technology becomes available, we were still able to see the ways in which demand, ownership, and resources effect ride sharing. The complexity of this subject continues to evolve, and thus the models do. This is reflected in the increasing complexity of our iterations. The topic of autonomous ride-sharing will continue to gain momentum into the future, and our model could continue to expand to reflect the complexity of the system. For example, we could take into account realitic city street models where the vehicles are constrained to driveable areas that have traffic restrictions. Also, we could use traffic data and distance to determine the optimal locations based on time, or model tradeoffs between time and profitability. We breifly analyzed a profitability model, but one could also develop and model certain parameters like the overall environmental impact of a car while it's doing a certain task. 